In [1]:
import pandas as pd
from lifelines import CoxPHFitter
from src.helper_methods import *
from src.pipe_store import *
from src.constants import *
from lifelines import CoxPHFitter

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

# Data Loading
### Deceased Donors

In [2]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'

donor_type ='Deceased'
# antibody_epitope = 
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df = df.assign(
                No_DESA = df['DESA'].apply(lambda x: 0 if x else 1),
                DESA_12 = df['#DESA'].apply(lambda x: 1 if (x >= 1) & (x <= 2) else 0),
                DESA_3ormore = df['#DESA'].apply(lambda x: 1 if x >= 3 else 0),
        )

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.018856s
- selceted cohort --> Donor Type: Deceased, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (3235, 23) | Computation Time: 0.010357s
Step: integer_encoder | Shape: (3235, 12) | Computation Time: 0.000919s
Step: set_time_event_label | Shape: (3235, 14) | Computation Time: 0.001632s
Step: eng_immunological_features | Shape: (3235, 19) | Computation Time: 0.028588s
Step: censoring_deaths | Shape: (3235, 19) | Computation Time: 0.001027s
Step: setting_prediction_horizon | Shape: (3235, 19) | Computation Time: 0.003418s


# Structural Causal Models

# Checking the Cox Proportional Hazard Assumptions

https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html?highlight=cox%20assumptions

The proportional hazard assumption is that all individuals have the same hazard function, but a unique scaling factor infront. So the shape of the hazard function is the same for all individuals, and only a scalar multiple changes per individual.

In [3]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
    # 'No_DESA', 
    # 'DESA_12',
    'DESA_3ormore'
]

In [9]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] >= 1]
df_early = df_total[df_total['T'] < 1]
df_class_I = df_total[df_total['Class_I'] == 1]
df_class_II = df_total[df_total['Class_II'] == 1]
df_class_I_II = df_total[df_total['Class_I_II'] == 1]

formula = ' + '.join(confounders)
# for df_l, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df_l = df_class_II
cph = CoxPHFitter()
cph.fit(df_l, duration_col='T', event_col='E', formula=formula)
len = df_l['DESA_3ormore'].sum()
print(f' Bad DESA size in data set {len}')
cph.print_summary()

 Bad DESA size in data set 67


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 114 total observations, 52 right-censored observations>
             duration col = 'T'
                event col = 'E'
      baseline estimation = breslow
   number of observations = 114
number of events observed = 62
   partial log-likelihood = -254.61
         time fit was run = 2022-04-07 21:00:04 UTC

---
                           coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                        
CIPHour_DBD                0.15       1.16       0.25            -0.35             0.64                 0.71                 1.91
CIPHour_DCD                0.34       1.41       0.24            -0.13             0.81                 0.88                 2.25
DESA_3ormore               0.57       1.78       0.29             0.00             1.14                 1.00                 3.14
DonorAge_NOTR             -1.02       0.36       0.61            -2.22             0.18                 0.11                 1.19
DonorAge_NOTR_power2       1.15       3.15       0.63            -0.08             2.38                 0.92                10.76
IL2rMoAb                  -0.25       0.78       0.42            -1.08             0.58                 0.34                 1.78
RecipientAge_NOTR          0.57       1.76       0.82            -1.05             2.18                 0.35                 8.87
RecipientAge_NOTR_power2  -0.57       0.57       0.84            -2.21             1.08                 0.11                 2.94

                           cmp to     z    p   -log2(p)
covariate                                              
CIPHour_DBD                  0.00  0.59 0.56       0.85
CIPHour_DCD                  0.00  1.43 0.15       2.71
DESA_3ormore                 0.00  1.98 0.05       4.37
DonorAge_NOTR                0.00 -1.67 0.09       3.41
DonorAge_NOTR_power2         0.00  1.83 0.07       3.90
IL2rMoAb                     0.00 -0.60 0.55       0.86
RecipientAge_NOTR            0.00  0.69 0.49       1.03
RecipientAge_NOTR_power2     0.00 -0.67 0.50       1.00
---
Concordance = 0.61
Partial AIC = 525.23
log-likelihood ratio test = 10.93 on 8 df
-log2(p) of ll-ratio test = 2.28

The take away message on Class I, Class II, and combination of both is as follow:
- The hazard for DESA >3 is the highest for Class I, then Class II, and finally Class I & II.


|Deceased donors (3235)| #Tx with DESA > 2 | DESA > 2 (HR) | 95% CI|
| ------------- | --------------| -----------|-----------|
| All           |         214|      2.41 | 1.98-2.94
| DESA Class I  |    113      |      2.07	  |  1.15-3.72
| DESA Class II |      67    |    1.78	   |  1.00-3.14
| DESA Class I & II|  34    |     1.49	| 0.11-20.27	
| Early failures (< 1 y)| 46 |  1.26  | 0.89-1.79
| Late failures (>1 y)| 168  |   2.64 |2.07-3.37

# Living Donors

In [10]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    # 'CIPHour_DBD', 'CIPHour_DCD', 
    # 'No_DESA', 
    # 'DESA_12',
    'DESA_3ormore'
]

In [12]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type ='Living'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'RecipientAge_NOTR',
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df = df.assign(
                No_DESA = df['DESA'].apply(lambda x: 0 if x else 1),
                DESA_12 = df['#DESA'].apply(lambda x: 1 if (x >= 1) & (x <= 2) else 0),
                DESA_3ormore = df['#DESA'].apply(lambda x: 1 if x >= 3 else 0),
        )
df.shape

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.014809s
- selceted cohort --> Donor Type: Living, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (1455, 23) | Computation Time: 0.006372s
Step: integer_encoder | Shape: (1455, 9) | Computation Time: 0.000724s
Step: set_time_event_label | Shape: (1455, 11) | Computation Time: 0.001982s
Step: eng_immunological_features | Shape: (1455, 16) | Computation Time: 0.015094s
Step: censoring_deaths | Shape: (1455, 16) | Computation Time: 0.00142s
Step: setting_prediction_horizon | Shape: (1455, 16) | Computation Time: 0.002177s


Variable DialysisYears does not exist in the data frame columns


(1455, 18)

In [15]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] >= 1]
df_early = df_total[df_total['T'] < 1]
df_class_I = df_total[df_total['Class_I'] == 1]
df_class_II = df_total[df_total['Class_II'] == 1]
df_class_I_II = df_total[df_total['Class_I_II'] == 1]

formula = ' + '.join(confounders)
# for df_l, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df_l = df_class_I_II
cph = CoxPHFitter()
cph.fit(df_l, duration_col='T', event_col='E', formula=formula)
len = df_l['DESA_3ormore'].sum()
print(f' Bad DESA size in data set {len}')
cph.print_summary()

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/__init__.py:1103: ConvergenceWarning: Column(s) ['DESA_3ormore'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1524: LinAlgWarning: Ill-conditioned matrix (rcond=0): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/__i

ConvergenceError: delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model

|Living donors (1455) | #Tx with DESA > | DESA > 2 (HR) | 95% CI|
| ------------- | --------------| -----------|-----------|
| All           | 80         |      1.38     | 0.87-2.18	
| DESA Class I  | 31         |        1.78    | 0.63-5.04	
| DESA Class II | 32         |     0.32	       | 0.08-1.26
| DESA Class I & II| 17      |            | 
| Early failures (< 1 y)| 6  |    4.28    | 1.42-12.92	
| Late failures (>1 y)| 74          |  1.24	  |0.74-2.10

# Potential Outcomes

## Finding Inverse Probability Wieghts (IPW)

In [36]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] > 1]
df_early = df_total[df_total['T'] < 1]
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
df_weight = find_ipw(df, confounders, treatments, scaler=None, verbose=False)

cph = CoxPHFitter()
cols = [
    # 'No_DESA',
    'Specific_DESA',
    'Other_DESA',
    'E', 'T', 
    'w'
]
cph.fit(
    df_weight[cols], 
    weights_col='w',
    duration_col='T', event_col='E', robust=True)
print(cph.print_summary())

['No_DESA ~ RecipientAge_NOTR + RecipientAge_NOTR_power2 + DonorAge_NOTR + DonorAge_NOTR_power2 + IL2rMoAb + CIPHour_DBD + CIPHour_DCD', 'Other_DESA ~ RecipientAge_NOTR + RecipientAge_NOTR_power2 + DonorAge_NOTR + DonorAge_NOTR_power2 + IL2rMoAb + CIPHour_DBD + CIPHour_DCD', 'Specific_DESA ~ RecipientAge_NOTR + RecipientAge_NOTR_power2 + DonorAge_NOTR + DonorAge_NOTR_power2 + IL2rMoAb + CIPHour_DBD + CIPHour_DCD']


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 9678.44 total observations, 5861.51 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 9678.44
number of events observed = 3816.93
   partial log-likelihood = -32960.71
         time fit was run = 2022-04-06 15:36:58 UTC

---
                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                             
Specific_DESA   1.16       3.19       0.12             0.92             1.40                 2.51                 4.05
Other_DESA      0.24       1.27       0.14            -0.03             0.51                 0.97                 1.67

                cmp to    z      p   -log2(p)
covariate                                    
Specific_DESA     0.00 9.49 <0.005      68.57
Other_DESA        0.00 1.74   0.08       3.60
---
Concordance = 0.54
Partial AIC = 65925.42
log-likelihood ratio test = 999.02 on 2 df
-log2(p) of ll-ratio test = 720.64

None


In [39]:
cph.check_assumptions(df_weight[cols], p_value_threshold=0.05, show_plots=True)

Proportional hazard assumption looks okay.


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/kaplan_meier_fitter.py:317: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  warnings.warn(
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/mixins.py:108: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  for variable in self.params_.index & (colu

[]

In [40]:
from lifelines.statistics import proportional_hazard_test
results = proportional_hazard_test(cph, df_weight[cols], time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/statistics.py:143: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return self.summary.to_latex()


<lifelines.StatisticalResult: proportional_hazard_test>
    time_transform = rank
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 9688.73 total observations, 5829.95 right-censored observations>
         test_name = proportional_hazard_test

---
               test_statistic    p  -log2(p)
Other_DESA               0.03 0.87      0.20
Specific_DESA            0.12 0.73      0.45

### Living Donor

In [52]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type ='Living'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        # 'TypeCadaveric_NOTR', 
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(features_from_antibody_epitopes)
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    # .pipe(feature_scaler, num_col, scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df['DESA_3more'] = df['#DESA'].apply(lambda x: 1 if x > 2 else 0)

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.01847s
- selceted cohort --> Donor Type: Living, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (1455, 23) | Computation Time: 0.013016s
Step: integer_encoder | Shape: (1455, 12) | Computation Time: 0.000888s
Step: set_time_event_label | Shape: (1455, 16) | Computation Time: 0.004486s
Step: censoring_deaths | Shape: (1455, 16) | Computation Time: 0.002584s
Step: setting_prediction_horizon | Shape: (1455, 16) | Computation Time: 0.005771s


In [53]:
df.sample()

,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,Donor_Type,IL2rMoAb,No_DESA,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,E,T,DESA_3more
838,0,{},20,2.7,44,0.0,0.0,Living,0,1,0,0,0,0,0,15.0,0


In [16]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] > 1]
df_early = df_total[df_total['T'] < 1]

# for df, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df = df_total
confounders = [
    'RecipientAge_NOTR', 'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 'DonorAge_NOTR*DonorAge_NOTR', 'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]

df_treat_group = create_treatment_grups(df, [RELEVANT_DESA_BAD])
# treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA', 'DESA_3more']
# df_weight = find_ipw(df_treat_group, confounders, treatments, verbose=False)
df_treat_group
# cph = CoxPHFitter()
# cols = [
#     'DonorAge_NOTR', 'RecipientAge_NOTR',
#     'DialysisYears', 'IL2rMoAb',
#     'CIPHour_DBD', 'CIPHour_DCD', 
#     'No_DESA',
#     'Specific_DESA',
#     'DESA_3more',
#     'E', 'T', 'w',
# ]
# cph.fit(df_weight[cols], weights_col='w', duration_col='T', event_col='E', robust=True)
# # print(title)
# cph.print_summary()


,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,TypeCadaveric_NOTR,Donor_Type,IL2rMoAb,...,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,E,T,DESA_3more,Specific_DESA,Other_DESA,Groups
0,0,{},44,2.3,54,9.000000,0.000000,0,Deceased,0,...,0,0,0,0,0,12.191781,0,0,0,1
1,0,{},54,2.1,32,21.750000,0.000000,0,Deceased,0,...,0,0,0,0,0,15.000000,0,0,0,1
2,0,{},44,4.9,27,36.000000,0.000000,0,Deceased,0,...,0,0,0,0,1,6.956164,0,0,0,1
3,0,{},23,0.0,40,30.000000,0.000000,0,Deceased,1,...,0,0,0,0,0,4.509589,0,0,0,1
4,0,{},55,5.4,38,0.000000,17.216667,1,Deceased,0,...,0,0,0,0,1,0.010959,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,0,{},51,1.5,43,29.000000,0.000000,0,Deceased,1,...,0,0,0,0,0,12.126027,0,0,0,1
4686,0,{},54,1.7,37,20.700000,0.000000,0,Deceased,1,...,0,0,0,0,1,10.879452,0,0,0,1
4687,0,{},21,3.5,46,17.183333,0.000000,0,Deceased,1,...,0,0,0,0,0,15.000000,0,0,0,1
4688,0,{},42,2.5,23,0.000000,32.183333,1,Deceased,1,...,0,0,0,0,1,11.816438,0,0,0,1
